In [1]:
from utils.classificaton_utils import binarize
from sklearn.model_selection import train_test_split
from flair.models.text_classification_model import TARSClassifier
from flair.datasets import SentenceDataset
from flair.data import Sentence, Corpus

In [2]:
import flair
flair.__version__

'0.7'

In [3]:
import sys
print(sys.version)

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]


In [4]:
lemma, pos = 'machine','NN'
senses = {'machine_nn01-38475835','machine_nn01-38475923'}
relations = ['seed','synonym'] # 'descendant','sibling'

train, val, test  = binarize(lemma,
                            pos, 
                            senses, 
                            relations,
                            strict_filter=True,
                            start=1700,
                            end=2000)

Index(['sense_id', 'lemma_definition', 'definition', 'word_id', 'lemma',
       'quotation_id', 'source', 'text', 'year'],
      dtype='object')
# senses before filtering by date = 517
# senses after filtering by date = 433


# of seed senses 26 
# of synonyms 383 
# of branch senses 0


# of seeds selected 2 
# of synonyms selected 10 
# of branches selected 0


In [5]:
print(train.shape,test.shape)

(127, 17) (107, 17)


In [6]:
train.head()

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses,full_text
0,machine_nn01-38476397,"A complex device, consisting of a number of in...",Originally U.S. The controlling organization o...,machine_nn01,machine,machine_nn01-38476417,"{'title': 'Let.', 'author': 'A. Burlingame', '...","{'keyword': 'machine', 'full_text': 'Our old p...",1860.0,0,machine_nn01-38476397,"{'end': None, 'start': 1860, 'obsolete': False...","[[machine_nn01-38476397, seed, machine_nn01]]",seed,{machine_nn01-38476397},{machine_nn01-38476397},Our old policy of ‘running with the Seward mac...
1,machine_nn01-38475046,"A complex device, consisting of a number of in...",An aircraft or airship. Cf. sense 6b.,machine_nn01,machine,machine_nn01-38475083,"{'title': 'Pastoral', 'author': '‘N. Shute’', ...","{'keyword': 'machine', 'full_text': 'The machi...",1944.0,0,machine_nn01-38475046,"{'end': None, 'start': 1896, 'obsolete': False...","[[machine_nn01-38475046, seed, machine_nn01]]",seed,{machine_nn01-38475046},{machine_nn01-38475046},The machine before them opened out and trundle...
2,machine_nn01-38474301,"A complex device, consisting of a number of in...",Theatre. A (usually movable) contrivance for t...,machine_nn01,machine,machine_nn01-38474380,"{'title': 'Vauxhall', 'author': 'T. Hood', 'ge...","{'keyword': 'machine', 'full_text': 'Time's ri...",1844.0,0,machine_nn01-38474301,"{'end': None, 'start': 1609, 'obsolete': False...","[[machine_nn01-38474301, seed, machine_nn01]]",seed,{machine_nn01-38474301},{machine_nn01-38474301},"Time's ripe for the Ballet, Like bees they all..."
3,machine_nn01-38476316,"A complex device, consisting of a number of in...",slang. The penis; the female genitals (rare). ...,machine_nn01,machine,machine_nn01-38476379,"{'title': 'Pop from Beginning', 'author': 'N. ...","{'keyword': 'machine', 'full_text': 'He had bl...",1969.0,0,machine_nn01-38476316,"{'end': None, 'start': 1749, 'obsolete': False...","[[machine_nn01-38476316, seed, machine_nn01]]",seed,{machine_nn01-38476316},{machine_nn01-38476316},He had black leather pants so tight that his m...
4,machine_nn01-38474607,"A complex device, consisting of a number of in...","A (usually wheeled) vehicle or conveyance, esp...",machine_nn01,machine,machine_nn01-38474785,"{'title': 'Lord Jim', 'author': 'J. Conrad', '...","{'keyword': 'machine', 'full_text': 'The littl...",1900.0,0,machine_nn01-38474607,"{'end': None, 'start': 1687, 'obsolete': False...","[[machine_nn01-38474607, seed, machine_nn01]]",seed,{machine_nn01-38474607},{machine_nn01-38474607},The little machine shook and rocked tumultuous...


In [7]:
train_sentences = [(row.text.get('full_text',''),
                    row.text.get('keyword',''),
                    row.label)
                        for i,row in train.iterrows()]
                
train_sentences = [Sentence(s + ' ' + t).add_label('machine_or_not',l) for s,t,l in train_sentences if s and t]


In [8]:
test_sentences = [(row.text.get('full_text',''),
                    row.text.get('keyword',''),
                    row.label)
                        for i,row in test.iterrows()]
                
test_sentences = [Sentence(s + ' ' + t).add_label('machine_or_not',l) for s,t,l in test_sentences if s and t]


In [9]:
train_sentences[10]

Sentence: "One thousand perfect copies. . could only have been daily produced at one press by the labour of two men . The machine produces sixteen thousand copies. machine"   [− Tokens: 28  − Sentence-Labels: {'machine_or_not': [0 (1.0)]}]

In [10]:
corpus = Corpus(train=train, test=test)

In [11]:
from flair.trainers import ModelTrainer

# 1. load base TARS
tars = TARSClassifier.load('tars-base')

# 2. make the model aware of the desired set of labels from the new corpus
tars.add_and_switch_to_new_task("machine_or_not", label_dictionary=corpus.make_label_dictionary())

# 3. initialize the text classifier trainer with your corpus
trainer = ModelTrainer(tars, corpus)

# 4. train model
trainer.train(base_path='resources/taggers/machine_or_not', # path to store the model artifacts
              learning_rate=0.02, # use very small learning rate
              mini_batch_size=1, # small mini-batch size since corpus is tiny
              max_epochs=10, # terminate after 10 epochs
              train_with_dev=True,
              )

2021-01-08 14:07:54,380 loading file /home/kbeelen/.flair/models/tars-base.pt


RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /opt/conda/conda-bld/pytorch_1579040055865/work/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /opt/conda/conda-bld/pytorch_1579040055865/work/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x47 (0x7f08a12a3627 in /data/anaconda/envs/py37torch/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 0x1f5b (0x7f08a5837e2b in /data/anaconda/envs/py37torch/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::unique_ptr<caffe2::serialize::ReadAdapterInterface, std::default_delete<caffe2::serialize::ReadAdapterInterface> >) + 0x57 (0x7f08a5838f77